In [8]:
import copernicusmarine
import xarray as xr

def retrieve_data(lats, longs, parameter):
    """
    Retrieve data from the Copernicus Marine server.

    Parameters
    ----------
    lats : tuple
        (min_lat, max_lat) for the region of interest.
    longs : tuple
        (min_lon, max_lon) for the region of interest.
    parameter : str
        Parameter to retrieve. Options:
        - "chl" : Chlorophyll
        - "sst" : Temperature ("thetao_mean")
        - "sal" : Salinity ("so_mean")
        - "ssh" : Sea Height ("zos_mean")
        - "mlt" : Mixed Layer Thickness ("mlotst_mean")
        - "eke" : Eddy Kinetic Energy ("uo_mean , vo_mean")

    Returns
    -------
    xarray.Dataset
        Dataset containing the requested variable for the specified region and time.
    """
    un="kcastello@ucsd.edu",
    p="2RVrJrqaeHQ@ue_"
    
    # If we want a pretty green chlorophyll plot we need to open a different dataset.
    if parameter == "chl":
        print("here")
        dataset_id = "cmems_mod_glo_bgc-pft_anfc_0.25deg_P1D-m"
        variables = ["chl"]
    else:
        dataset_id = "cmems_mod_glo_phy-mnstd_my_0.25deg_P1D-m"
        parameter_map = {
            "sst": "thetao_mean",
            "sal": "so_mean",
            "ssh": "zos_mean",
            "mlt": "mlotst_mean",
            "eke": ["uo_mean", "vo_mean"]}
        if parameter not in parameter_map:
            raise ValueError(f"Invalid parameter '{parameter}'. Choose from {list(parameter_map.keys())}")
        variables = parameter_map[parameter]
    if isinstance(variables, str):
        variables = [variables]
    ds = copernicusmarine.open_dataset( dataset_id=dataset_id, variables=variables, minimum_latitude=lats[0], 
                                        maximum_latitude=lats[1], minimum_longitude=longs[0], maximum_longitude=longs[1],
                                      username=un, password=p)

    # Compute Eddy Kinetic Energy if 'eke' is selected
    if parameter == "eke":
        u = ds["uo_mean"]
        v = ds["vo_mean"]
        eke = 0.5 * (u**2 + v**2)
        ds = eke.to_dataset(name="eke")
    return ds    

In [9]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from matplotlib.offsetbox import AnchoredText
from matplotlib.animation import FuncAnimation, PillowWriter
import numpy as np
import hvplot.xarray
import panel as pn

pn.extension('bokeh')

def show_background_animation_autoplay(ds, parameter, duration=5):
    """
    Show an auto-playing hvPlot animation.

    Args:
        ds (xarray.Dataset): Dataset from retrieve_data().
        parameter (str): The parameter to plot.
        duration (float): Total time in seconds for one full loop.
    """
    # Create interactive plot with hvPlot
    plot = ds[parameter].hvplot.image(
        x='longitude',
        y='latitude',
        cmap='viridis',
        width=750,
        height=250,
        clim=(float(ds[parameter].min()), float(ds[parameter].max())),
        geo=True,
        project=True,
        coastline=True,
        rasterize=True,
        dynamic=True,
        groupby='time'
    )

    # Wrap plot in Panel to access widgets
    pn_plot = pn.panel(plot)

    # Access the time slider widget if it exists
    widgets = pn_plot.widgets
    if 'time' in widgets:
        time_widget = widgets['time']
        time_widget.player.loop_policy = 'loop'
        time_widget.player.interval = int(duration * 1000 / len(ds['time']))

    return pn_plot

In [3]:
data = retrieve_data((10, 20), (10, 20), "sst")
show_background_animation_autoplay(ds, "thetao_mean", duration=60)

INFO - 2025-08-20T00:31:23Z - Selected dataset version: "202311"
INFO - 2025-08-20T00:31:23Z - Selected dataset part: "default"


NameError: name 'show_background_animation_hvplot' is not defined